In [28]:
import nltk
from nltk import tokenize
from nltk.collocations import *
from bs4 import BeautifulSoup
from collections import Counter
import os  
from elasticsearch import Elasticsearch


#tokenizer = nltk.RegexpTokenizer(r'\w+')
tokenizer = nltk.WhitespaceTokenizer()

allData = {}

for fn in os.listdir('troonredes'):
    with open('troonredes/' + fn) as troonrede:
        soup = BeautifulSoup(troonrede, 'lxml')
        text = soup.find("div", {"id": "post-content"}).text
        html = unicode(''.join([str(p) for p in soup.find("div", {"id": "post-content"}).find_all('p')]), errors='ignore')
        headOfState = soup.find("a", {"rel": "category tag"}).text
        title = soup.title.text.strip()[:-16]
        year = int(soup.find("h1", {"class": "post-title"}).text[-4:])
        sentences = [tokenizer.tokenize(sent) for sent in nltk.sent_tokenize(text)]
        if headOfState not in allData.keys():
            allData[headOfState] = {}
        documentData = {}
        documentData['full text'] = text
        documentData['sentences'] = sentences
        documentData['html'] = html
        documentData['year'] = year
        allData[headOfState][title] = documentData

In [29]:
import pandas as pd

dataList = []
for staatshoofd, troonrede in allData.iteritems():
    for key, value in troonrede.iteritems():
        l = value.values()
        l.append(key)
        l.append(staatshoofd)
        dataList.append(l)

df = pd.DataFrame(dataList)
df.columns = ['full text', 'html', 'year', 'sentences', 'title', 'head of state']
df = df.sort_values('year').reset_index(drop=True)

es = Elasticsearch()

df.head()

,full text,html,year,sentences,title,head of state
0,\nEdel Mogende Heeren!\nIn den loop van dit ja...,<p>Edel Mogende Heeren!</p><p>In den loop van ...,1818,"[[Edel, Mogende, Heeren!], [In, den, loop, van...",Troonrede van 19 oktober 1818,Willem I
1,"\nEdel Mogende Heeren!\nDe Koning heeft ons, d...",<p>Edel Mogende Heeren!</p><p>De Koning heeft ...,1819,"[[Edel, Mogende, Heeren!], [De, Koning, heeft,...",Openingsrede van 18 oktober 1819,Willem I
2,\nEdel Mogende Heeren!\nSedert de laatste reiz...,<p>Edel Mogende Heeren!</p><p>Sedert de laatst...,1820,"[[Edel, Mogende, Heeren!], [Sedert, de, laatst...",Troonrede van 16 oktober 1820,Willem I
3,\nEdel Mogende Heeren!\nHet strekt mij tot een...,<p>Edel Mogende Heeren!</p><p>Het strekt mij t...,1821,"[[Edel, Mogende, Heeren!], [Het, strekt, mij, ...",Troonrede van 15 oktober 1821,Willem I
4,\nEdel Mogende Heeren!\nBij het openen uwer ve...,<p>Edel Mogende Heeren!</p><p>Bij het openen u...,1822,"[[Edel, Mogende, Heeren!], [Bij, het, openen, ...",Troonrede van 21 oktober 1822,Willem I


In [30]:
dfIter = list(df[['full text', 'head of state', 'year', 'sentences', 'title','html']].iterrows())
for row in dfIter:
    ID = row[0] + 1
    es.index(index="troonredes", doc_type="troonrede", id=ID, \
             body={"speaker" : row[1]['head of state'], "year" : row[1]['year'],
                   "text" : row[1]['full text'], "sentences" : row[1]['sentences'],
                   "title" : row[1]['title'], "html" : row[1]['html']})